# Migrate to Ipyleaflet

**In this notebook we explain how to migrate from `HEREMapInspector` to `IpyleafletInspector`.**

`here-inspector` is promoting `HEREv3.reducedDay` as default tile provider. If `options.api_key` is not set, calls to `inspect()` use tiles from `Esri.WorldTopoMap` and instancations of `HEREMapInspector` raise a `ValueError`.

In 2.28.0 release, `HEREMapInspector` has been adapted to use the `ipyleaflet` package so that the only differences between `IpyleafletInspector` and `HEREMapInspector` are about the default configuration:

|  | `HEREMapInspector` | `IpyleafletInspector` |
|:-----------|:-------------------|:----------------------|
| HERE api key | needed by default | not needed by default |
|  default<br/>`xyzservices`<br/>`TileProvider` |  `HEREv3.reducedDay`     | `Esri.WorldTopoMap` |


Note however that you still need a HERE api key to be able to use `IpyleafletInspector` with `HEREv3` tile providers.

The cells below demonstrate how to use `IpyleafletInspector` instead of `HEREMapInspector` with only minimal changes. It replicates the cells from `ExploreInspector_HereMapWidgetForJupyter.ipynb` using a `HERMapInspector` instance.

In [ ]:
# First, load GeoJSON features and shapely geometries:

import json
from shapely.geometry import shape, Point, MultiPoint, LineString, Polygon

with open("sample_datasets/FRA.geo.json") as f:
    fra = json.load(f)["features"][0] 

pt1 = Point(30, 50)
pt2 = Point(32, 49)
pt3 = Point(31, 52)

ls1 = LineString([(29, 49), (30, 51), (31, 49), (32, 50), (33, 51)])
ls2 = LineString([(30, 47), (31, 48), (29, 49), (31, 50), (29, 51)])

poly1 = Polygon([(28, 46), (28, 52), (29, 53), (31, 53), (33, 48), (32, 47), (28, 46)])
poly2 = MultiPoint([(40, 51), (43, 49)]).buffer(3)

In [ ]:
# Configure your HERE API key
# HERE API key is mandatory to use HEREv3 tiles.

import os
from here.inspector import options

options.api_key = os.getenv("LS_API_KEY") or "YOUR-ACTUAL-API-KEY" 

# `import as`

Both `HEREMapInspector` and `IpyleafletInspector` share the same interface. You can continue to fine-tune an instance of `IpyleafletInspector` as if it were an instance `HEREMapInspector`. 
Notebooks using `HEREMapInspector` can be migrated to `IpyleafletInspector` by replacing class names:

Replace

```Python
from here.inspector import HEREMapInspector 
```
with:

```Python
from here.inspector import IpyleafletInspector as HEREMapInspector 
```

In [ ]:
# Instantiate IpyleafletInspector as if it were a HERE Map Widget for Jupyter based inspector
from here.inspector import IpyleafletInspector as HEREMapInspector 
from here.inspector import here_basemaps

inspector = HEREMapInspector()
inspector.set_basemap(basemap=here_basemaps.normalDay)
inspector.set_zoom(14)
inspector.set_center(Point(13.4083, 52.5186))
inspector.show()

In [ ]:
from here.inspector import external_basemaps

inspector = HEREMapInspector()
inspector.set_basemap(basemap=external_basemaps.OpenStreetMap.Mapnik)
inspector.show()

In [ ]:
inspector = HEREMapInspector()

# Add features with HERE Map Widget for Jupyter specific styles
inspector.add_features(
    fra, name="France",
    style={"fillColor": "rgba(245, 176, 65, 0.5)", "strokeColor": "black"},
    hover_style={'fillColor': "rgb(0,255,255,0.5)"}
)
inspector.add_features(
    [pt1, pt2, pt3, ls1, poly2], 
    name="Some geometries",
    style={'strokeColor': 'magenta', 'fillColor': "rgba(245, 176, 65, 0.5)"},
    hover_style={'fillColor': "rgb(255,255,0,0.8)"},
    point_style={'radius': 8}
)

# Set a different base map the same way as you do on HEREMapInspector
inspector.set_basemap(external_basemaps.Esri.WorldImagery)

inspector.show()

In [ ]:
inspector = HEREMapInspector()

basemap = {
    'name': 'U.S. Geological Survey',
    'url': 'https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}',
    'maxZoom': 20,
    'attribution': 'Tiles courtesy of the <a href="https://usgs.gov/">U.S. Geological Survey</a>'
}

inspector.set_basemap(basemap)
inspector.show()

# `backend` function

The `backend()` method can further be used to access the underlying backend. Since release 2.28.0, both `HEREMapInspector` and `IpyleafletInspector` use the same `ipyleaflet` `Map` object backend.

In [ ]:
from here.inspector import HEREMapInspector as HEREMapInspectorDeprecated 
from here.inspector import IpyleafletInspector

ipyleaflet_inspector = IpyleafletInspector()
depecrated_heremap_inspector = HEREMapInspectorDeprecated()

(type(ipyleaflet_inspector.backend()), 
 type(depecrated_heremap_inspector.backend())
)

### Example of a custom base map

<!--
from urllib.parse import quote, unquote
svg = """<svg width="40" height="48" viewBox="0 0 40 48" fill="none" xmlns="http://www.w3.org/2000/svg"><path class="triangle" d="M11,36.8l-5.5,5.5L0,36.8H11z" fill="#48DAD0"></path><path class="HERE" d="M19.1,24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9,1.7-0.5,2.7,0.5L19.1,24.2z M30.9,9.2c0.9-0.9,1.7-0.5,2.7,0.5
    L31.3,12C30.1,10.7,30.2,9.9,30.9,9.2z M38,11.6c-1.1,1.6-2.9,4.2-4.9,2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4
	c-1.6,1.6-2,3.4-1.5,5.1l-1.6-1.8c-0.5,0.3-2.4,1.9-0.9,4.5l-1.8-1.5l-2.4,2.4l3.2,3.2c-2.5-1.9-5.1-1.8-7.2,0.4
	c-2.3,2.3-2.1,5-0.4,7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9,0.9-1.5,2.2-1.3,3.1L4,24.6l-2.6,2.6l9.6,9.6h5.2l-3.5-3.5
	c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8,1.8-0.3,3.6,1.4l3.4,3.4l2.6-2.6l-3.2-3.2c2.3,1.8,5.1,1.9,7.7-0.6l0,0c1.5-1.4,2-2.8,2-2.8
	l-1.9-1.3c-1.1,1.6-2.9,4.2-4.9,2.3l5-5l3.1,3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5,0-4.1c0.4,0.7,0.9,1.5,1.5,2.1
	c2.5,2.5,5.7,3,8.6,0.2l0,0c1.5-1.4,2-2.8,2-2.8S38,11.6,38,11.6z" fill="#000000"></path></svg>
"""
print(f"""![HERE](data:image/svg+xml,{quote(svg)})
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>""")
-->
![HERE](data:image/svg+xml,%3Csvg%20width%3D%2240%22%20height%3D%2248%22%20viewBox%3D%220%200%2040%2048%22%20fill%3D%22none%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20class%3D%22triangle%22%20d%3D%22M11%2C36.8l-5.5%2C5.5L0%2C36.8H11z%22%20fill%3D%22%2348DAD0%22%3E%3C/path%3E%3Cpath%20class%3D%22HERE%22%20d%3D%22M19.1%2C24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5L19.1%2C24.2z%20M30.9%2C9.2c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5%0A%20%20%20%20L31.3%2C12C30.1%2C10.7%2C30.2%2C9.9%2C30.9%2C9.2z%20M38%2C11.6c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4%0A%09c-1.6%2C1.6-2%2C3.4-1.5%2C5.1l-1.6-1.8c-0.5%2C0.3-2.4%2C1.9-0.9%2C4.5l-1.8-1.5l-2.4%2C2.4l3.2%2C3.2c-2.5-1.9-5.1-1.8-7.2%2C0.4%0A%09c-2.3%2C2.3-2.1%2C5-0.4%2C7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9%2C0.9-1.5%2C2.2-1.3%2C3.1L4%2C24.6l-2.6%2C2.6l9.6%2C9.6h5.2l-3.5-3.5%0A%09c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8%2C1.8-0.3%2C3.6%2C1.4l3.4%2C3.4l2.6-2.6l-3.2-3.2c2.3%2C1.8%2C5.1%2C1.9%2C7.7-0.6l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8%0A%09l-1.9-1.3c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.3l5-5l3.1%2C3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5%2C0-4.1c0.4%2C0.7%2C0.9%2C1.5%2C1.5%2C2.1%0A%09c2.5%2C2.5%2C5.7%2C3%2C8.6%2C0.2l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8S38%2C11.6%2C38%2C11.6z%22%20fill%3D%22%23000000%22%3E%3C/path%3E%3C/svg%3E%0A)
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>